In [2]:
import os
import numpy    as np
import pandas   as pd
import psycopg2 as psy
import itertools

from datetime import datetime as dt
from random   import randrange
from random   import sample
from random   import choice

/usr/local/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
#''' Check if file exists on path. Move existing file to ARCH. Create new input for service_dim.'''


In [4]:
# Setup script input parameters

min_obs = 5                      # minimal number of observation in output dataset
max_obs = 20                     # maximal number of observation in output dataset

obs = randrange(min_obs,max_obs) # number of observation
obs

15

In [5]:
main_path    = "Documents/CallCenterStaffing"
path_to_arch = "Arch/office"
path_to_file = "Input"
path_to_dict = "Dict"
file_name    = "service_dim.csv"
service_dict = "service_dict.csv"

now          = dt.now()

path_to_file = os.path.join(os.path.expanduser("~"),main_path,path_to_file,file_name)
path_to_arch = os.path.join(os.path.expanduser("~"),main_path,path_to_arch)
service_dict = os.path.join(os.path.expanduser("~"),main_path,path_to_dict,service_dict)


def is_path_not_correct(path):
    return not(os.path.exists(path))

def is_path_correct(path):
    return os.path.exists(path)

if is_path_not_correct(path_to_file):
    raise Exception("Input file doesn't exist")

if is_path_not_correct(path_to_arch):
    raise Exception("Archive directory doesn't exist")

if is_path_not_correct(service_dict):
    raise Exception("Service dictionary doesn't exist")

time_stamp = "_" + str(now)[:10]
arch_file_name = 'arch_' + file_name.replace(".csv",time_stamp + ".csv")
 
os.rename(path_to_file, os.path.join(path_to_arch, arch_file_name))

Exception: Service dictionary doesn't exist

In [11]:
# connection to database and return list of column column_list
try:
    connection = psy.connect( user = "szymonbocian",
                              password = "",
                              host = "localhost",
                              port = "5432",
                              database = "dwh_call_center")

    cursor = connection.cursor()

    query = "SELECT column_name FROM INFORMATION_SCHEMA.columns WHERE table_schema = 'stg' AND table_name = 'service_dim';"
    cursor.execute(query)
    column_list = cursor.fetchall()
    
except (Exception, psy.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
finally:
        if(connection):
            cursor.close()
            connection.close()

column_list = [''.join(c for c in s if c not in '(),') for s in column_list]

print(column_list)

['service_full_name_eng', 'service_full_name_pl', 'service_short_name_eng', 'service_short_name_pl', 'service_code', 'service_business_key']


In [7]:
ENG_1 = ['Junior consultant ','Consultant ','Senior consultant ','Lead consultant ','Manager ','Contractor ']
ENG_2 = ['information ','solution ','help ']
ENG_3 = ['call','service','consultation','advice','connection']

PL_1 = ['Rozmowa ','Usługa ','Konsultancja ','Porada ','Połącznie ']
PL_2 = ['informacyjna ','ws rozwiązania ','ws pomocy ']
PL_3 = ['młodszego konsultanta','konsultanta','starszego konsultanta','głównego konsultanta','manadżera','kontraktora']

In [9]:
# Cartesian product of service name both in Polish and English

pl_service  = list()
eng_service = list()

for i in itertools.product(ENG_1,ENG_2,ENG_3):
    eng_service.append(i)

for i in itertools.product(PL_1,PL_2,PL_3):
    pl_service.append(i)

df = pd.DataFrame({
    "serviceFullNameEng" : [a[0] + a[1] + a[2] for a in eng_service],
    "serviceFullNamePl"  : [b[0] + b[1] + b[2] for b in pl_service]
})

df['serviceShortNameEng'] = [ i[0:4] + j[0:4].title() + k[0:4].title() for i,j,k in eng_service]
df['serviceShortNamePl']  = [ i[0:4] + j[0:4].title() + k[0:4].title() for i,j,k in pl_service]

df['serviceCode'] = [ i[0:6].upper() + j[0:4].upper() + k[0:4].upper() + '00' for i,j,k in eng_service]

sfnp = ('000' + (df.serviceFullNamePl.str.len()).astype(str))
df['sfnp'] = [s[-4:] for s in sfnp]

# Create service business key
df['serviceBusinessKey'] = [ i[0:1].upper() + j[0:1].upper() + k[0:4].upper() for i,j,k in eng_service] + df.sfnp

df = df.drop(['sfnp'], axis = 1)

df

,serviceFullNameEng,serviceFullNamePl,serviceShortNameEng,serviceShortNamePl,serviceCode,serviceBusinessKey
0,Junior consultant information call,Rozmowa informacyjna młodszego konsultanta,JuniInfoCall,RozmInfoMłod,JUNIORINFOCALL00,JICALL0042
1,Junior consultant information service,Rozmowa informacyjna konsultanta,JuniInfoServ,RozmInfoKons,JUNIORINFOSERV00,JISERV0032
2,Junior consultant information consultation,Rozmowa informacyjna starszego konsultanta,JuniInfoCons,RozmInfoStar,JUNIORINFOCONS00,JICONS0042
3,Junior consultant information advice,Rozmowa informacyjna głównego konsultanta,JuniInfoAdvi,RozmInfoGłów,JUNIORINFOADVI00,JIADVI0041
4,Junior consultant information connection,Rozmowa informacyjna manadżera,JuniInfoConn,RozmInfoMana,JUNIORINFOCONN00,JICONN0030
...,...,...,...,...,...,...
85,Contractor help call,Połącznie ws pomocy konsultanta,ContHelpCall,PołąWs PKons,CONTRAHELPCALL00,CHCALL0031
86,Contractor help service,Połącznie ws pomocy starszego konsultanta,ContHelpServ,PołąWs PStar,CONTRAHELPSERV00,CHSERV0041
87,Contractor help consultation,Połącznie ws pomocy głównego konsultanta,ContHelpCons,PołąWs PGłów,CONTRAHELPCONS00,CHCONS0040
88,Contractor help advice,Połącznie ws pomocy manadżera,ContHelpAdvi,PołąWs PMana,CONTRAHELPADVI00,CHADVI0029


In [12]:
# Create csv file with given destination

df.columns = column_list
df = df.sample(n = obs, random_state = 123)
df.to_csv(path_to_file, index = None, header = True)